In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [ ]:
import warnings
# warnings.filterwarnings('ignore')

## Reflect Tables into SQLALchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# View all of the classes that automap found
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()


In [ ]:
session = Session(engine)

In [ ]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route("/")
def welcome():
   return (
       f"Welcome to the Hawaii Climate Analysis API!<br/>"
       f"Available Routes:<br/>"
       f"/api/v1.0/precipitation<br/>"
       f"/api/v1.0/stations<br/>"
       f"/api/v1.0/tobs<br/>"
       f"/api/v1.0/temp/start/end"
   )
 @app.route("/api/v1.0/precipitation")
 def precipitation():
#    """Return the precipitation data for the last year"""
    # Calculate the date 1 year ago from last date in database
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    # Query for the date and precipitation for the last year
    precipitation = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= prev_year).all()
    # Dict with date as the key and prcp as the value
    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip)
 @app.route("/api/v1.0/stations")
 def stations():
   """Return a list of stations."""
    results = session.query(Station.station).all()
#    # Unravel results into a 1D array and convert to a list
    stations = list(np.ravel(results))
    return jsonify(stations=stations)
 @app.route("/api/v1.0/tobs")
 def temp_monthly():
    """Return the temperature observations (tobs) for previous year."""
    # Calculate the date 1 year ago from last date in database
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    # Query the primary station for all tobs from the last year
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= prev_year).all()
    # Unravel results into a 1D array and convert to a list
    temps = list(np.ravel(results))
    # Return the results
    return jsonify(temps=temps)
 @app.route("/api/v1.0/temp/<start>")
 @app.route("/api/v1.0/temp/<start>/<end>")
 def stats(start=None, end=None):
    """Return TMIN, TAVG, TMAX."""
    # Select statement
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    if not end:
        # calculate TMIN, TAVG, TMAX for dates greater than start
        results = session.query(*sel).\
            filter(Measurement.date >= start).all()
        # Unravel results into a 1D array and convert to a list
        temps = list(np.ravel(results))
        return jsonify(temps)
    # calculate TMIN, TAVG, TMAX with start and stop
    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
    # Unravel results into a 1D array and convert to a list
    temps = list(np.ravel(results))
    return jsonify(temps=temps)
 if __name__ == '__main__':
    app.run()

In [10]:
#measurement tobs
# session.query(Measurement.tobs).all()

In [11]:
# Station=Base.classes.station
# session.query(Station.name).all()

In [ ]:
data = engine.execute("SELECT * FROM station")
data.fetchall()

In [ ]:
from flask import Flask, jsonify

In [ ]:
# Station_list = [
#  {"station": "USC00519397", "name": "WAIKIKI 717.2, HI US", "latitude": 21.2716, "longitude": '-157.8168', "elevation": 3.0},
#  ("station": 'USC00513117', "name": 'KANEOHE 838.1, HI US', "latitude":21.4234, "longitude": -157.8015, "elevation": 14.6},
#  ("longitude": "station":'USC00514830', "name": 'KUALOA RANCH HEADQUARTERS, 886.9, HI US', "latitude":21.5213, "longitude" -157.8374, "elevation":7.0],
#  ("station":'USC00517948', "name": 'PEARL CITY, HI US', "latitude":21.3934, "longitude" -157.9751, "elevation":11.9},
#  ("station":'USC00518838', "name": 'UPPER WAHIAWA 874.3, HI US', "latitude":21.4992, "longitude" -158.0111, "elevation":306.6},
#  ("station":'USC00519523', "name": 'WAIMANALO EXPERIMENTAL FARM, HI US', "latitude":21.33556, "longitude" -157.71139, "elevation":19.5},
#  ("station":'USC00519281', "name": 'WAIHEE 837.5, HI US', "latitude":21.45167, "longitude" -157.84888999999998, "elevation":32.9},
#  ("longitude", "station":'USC00511918', "name": 'HONOLULU OBSERVATORY 702.2, HI US', "latitude":21.3152, "longitude" -157.9992, "elevation":0.9},
#  ("station":'USC00516128', "name": 'MANOA LYON ARBO 785.2, HI US', "latitude":21.3331, "longitude" -157.8025, "elevation":152.4}
# ]

In [ ]:
# Flask set up
# app = Flask(__name__)

In [ ]:

# @app.route("/api/v1.0/station")
# def station():
#     return jsonify(station)

In [110]:
# @app.route("/jsonified")
# def jsonified():
#     return jsonify(hello_dict)

In [ ]:
# if __name__ == "__main__":
#     app.run(debug=True)

## Bonus Challenge Assignment: Temperature Analysis II

In [120]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# For example
print(calc_temps('2012-02-28', '2012-03-05'))

[(62.0, 69.57142857142857, 74.0)]


In [121]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set

In [122]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for bar height (y value)
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

### Daily Rainfall Average

In [123]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation


In [124]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

# For example
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

In [125]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip
start_date = '2017-08-01'
end_date = '2017-08-07'

# Use the start and end date to create a range of dates


# Strip off the year and save a list of strings in the format %m-%d


# Use the `daily_normals` function to calculate the normals for each date string 
# and append the results to a list called `normals`.


In [126]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index

In [127]:
# Plot the daily normals as an area plot with `stacked=False`

## Close Session